In [1]:
import cv2
import numpy as np

def cartoonize_bilateral(img: np.ndarray,
                         d: int, sigmaColor: int, sigmaSpace: int,
                         low: int, high: int, median_k: int) -> np.ndarray:
    """Traditional cartoonize process using Bilateral filter and Canny."""
    
    # Apply Bilateral Filter for smoothing while preserving edges.
    sm = cv2.bilateralFilter(img, d, sigmaColor, sigmaSpace)
    
    # Convert image to grayscale for Canny edge detection.
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply Median Blur to reduce noise before edge detection.
    g = cv2.medianBlur(g, median_k)
    
    # Detect edges using Canny.
    edges = cv2.Canny(g, low, high)
    
    # Invert the edge map (black edges on white background).
    inv = cv2.bitwise_not(edges)
    
    # Convert the inverted edge map back to 3 channels for merging.
    invc = cv2.cvtColor(inv, cv2.COLOR_GRAY2BGR)
    
    # Combine the smoothed image with the edge mask.
    return cv2.bitwise_and(sm, invc)

In [2]:
import cv2
import numpy as np

def cartoonize_guided_kmeans(img: np.ndarray,
                             radius: int = 7, eps: int = 500, k: int = 16,
                             median_k: int = 5, low: int = 80, high: int = 220) -> np.ndarray:
    """Cartoonize process using Guided Filter and K-Means."""

    # 1. Edge Detection
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    g = cv2.medianBlur(g, median_k)
    edges = cv2.Canny(g, low, high)
    edges_inverted = cv2.bitwise_not(edges)
    edges_bgr = cv2.cvtColor(edges_inverted, cv2.COLOR_GRAY2BGR)

    # 2. Guided Filter
    # Check if opencv-contrib-python is installed
    if not hasattr(cv2, 'ximgproc'):
        raise ImportError("opencv-contrib-python is not installed. Install it with 'pip install opencv-contrib-python'.")
    image_guided = cv2.ximgproc.guidedFilter(img, img, radius, eps)

    # 3. K-Means Clustering for Color Quantization
    # Reshape the image to be a list of pixels
    pixels = image_guided.reshape((-1, 3))
    pixels = np.float32(pixels)

    # Define criteria, number of clusters(k) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    attempts = 10
    compactness, labels, centers = cv2.kmeans(pixels, k, None, criteria, attempts, cv2.KMEANS_RANDOM_CENTERS)

    # Convert back to uint8 and make the original image
    centers = np.uint8(centers)
    image_kmeans = centers[labels.flatten()]
    image_kmeans = image_kmeans.reshape(img.shape)

    # 4. Combination
    # Combine the K-Means image with the edge mask
    return cv2.bitwise_and(image_kmeans, edges_bgr)

In [3]:
import cv2
import numpy as np
from pathlib import Path
import os

# ==============================================================================
# == FUNCTIONS ==
# ==============================================================================

def cartoonize_bilateral(img: np.ndarray,
                         d: int, sigmaColor: int, sigmaSpace: int,
                         low: int, high: int, median_k: int) -> np.ndarray:
    """Traditional cartoonize process using Bilateral filter and Canny."""
    # Apply Bilateral Filter for smoothing while preserving edges.
    sm = cv2.bilateralFilter(img, d, sigmaColor, sigmaSpace)
    # Convert image to grayscale for Canny edge detection.
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply Median Blur to reduce noise before edge detection.
    g = cv2.medianBlur(g, median_k)
    # Detect edges using Canny.
    edges = cv2.Canny(g, low, high)
    # Invert the edge map (black edges on white background).
    inv = cv2.bitwise_not(edges)
    # Convert the inverted edge map back to 3 channels for merging.
    invc = cv2.cvtColor(inv, cv2.COLOR_GRAY2BGR)
    # Combine the smoothed image with the edge mask.
    return cv2.bitwise_and(sm, invc)

def cartoonize_guided_kmeans(img: np.ndarray,
                             radius: int = 7, eps: int = 500, k: int = 16,
                             median_k: int = 5, low: int = 80, high: int = 220) -> np.ndarray:
    """Cartoonize process using Guided Filter and K-Means."""
    # 1. Edge Detection
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    g = cv2.medianBlur(g, median_k)
    edges = cv2.Canny(g, low, high)
    edges_inverted = cv2.bitwise_not(edges)
    edges_bgr = cv2.cvtColor(edges_inverted, cv2.COLOR_GRAY2BGR)

    # 2. Guided Filter
    # Check if opencv-contrib-python is installed
    if not hasattr(cv2, 'ximgproc'):
        raise ImportError("opencv-contrib-python is not installed. Install it with 'pip install opencv-contrib-python'.")
    image_guided = cv2.ximgproc.guidedFilter(img, img, radius, eps)

    # 3. K-Means Clustering
    pixels = image_guided.reshape((-1, 3))
    pixels = np.float32(pixels)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    attempts = 10
    compactness, labels, centers = cv2.kmeans(pixels, k, None, criteria, attempts, cv2.KMEANS_RANDOM_CENTERS)
    centers = np.uint8(centers)
    image_kmeans = centers[labels.flatten()]
    image_kmeans = image_kmeans.reshape(img.shape)

    # 4. Combination
    return cv2.bitwise_and(image_kmeans, edges_bgr)

# ==============================================================================
# == BATCH PROCESSING ==
# ==============================================================================

# 1. Define Paths (Assuming 'photo_40' is in the same directory as the notebook)
input_folder = Path("photo_40")  # Updated input folder for 40 photos
output_bilateral_folder = Path("output_bilateral_40") # New output folder
output_guided_folder = Path("output_guided_40")       # New output folder

# 2. Create Output Directories
output_bilateral_folder.mkdir(exist_ok=True)
output_guided_folder.mkdir(exist_ok=True)

# 3. Define Parameters
bilateral_params_v10 = {"d": 5, "sigmaColor": 100, "sigmaSpace": 100,
                        "low": 80, "high": 220, "median_k": 5}
guided_params_k16 = {"radius": 7, "eps": 500, "k": 16, # Added k=16 explicitly
                     "median_k": 5, "low": 80, "high": 220}

# 4. Process Photos
# Get only .jpg files
image_files = list(input_folder.glob("*.jpg"))

# Optional: Sort files numerically if needed (handles 1.jpg, 10.jpg correctly)
def get_number(file_path):
    try:
        return int(os.path.splitext(file_path.name)[0])
    except ValueError:
        return -1 # Handle cases where filename is not a number

image_files = sorted(image_files, key=get_number)

print(f"Found files: {[f.name for f in image_files]}") # Check found files

for img_path in image_files:
    print(f"Processing: {img_path.name}...")
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"  -> ERROR: Could not read {img_path.name}.")
        continue

    try:
        # Apply and Save Bilateral
        out_bilateral = cartoonize_bilateral(img, **bilateral_params_v10)
        save_path_b = output_bilateral_folder / f"{img_path.stem}_bilateral.png"
        cv2.imwrite(str(save_path_b), out_bilateral)

        # Apply and Save Guided+KMeans (k=16)
        out_guided = cartoonize_guided_kmeans(img, **guided_params_k16)
        save_path_g = output_guided_folder / f"{img_path.stem}_guided_k16.png" # Updated save name
        cv2.imwrite(str(save_path_g), out_guided)

        print(f"  -> Completed: {img_path.name}")

    except Exception as e:
        print(f"  -> ERROR: An error occurred while processing {img_path.name}: {e}")

print("\nAll processing completed! Results saved to 'output_bilateral_40' and 'output_guided_40' folders.")

Found files: ['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '40.jpg']
Processing: 1.jpg...
  -> Completed: 1.jpg
Processing: 2.jpg...
  -> Completed: 2.jpg
Processing: 3.jpg...
  -> Completed: 3.jpg
Processing: 4.jpg...
  -> Completed: 4.jpg
Processing: 5.jpg...
  -> Completed: 5.jpg
Processing: 6.jpg...
  -> Completed: 6.jpg
Processing: 7.jpg...
  -> Completed: 7.jpg
Processing: 8.jpg...
  -> Completed: 8.jpg
Processing: 9.jpg...
  -> Completed: 9.jpg
Processing: 10.jpg...
  -> Completed: 10.jpg
Processing: 11.jpg...
  -> Completed: 11.jpg
Processing: 12.jpg...
  -> Completed: 12.jpg
Processing: 13.jpg...
  -> Completed: 13.jpg
Processing: 14.jpg...
  -> C